In [1]:
from hepmc import *
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
%matplotlib notebook

In [2]:
D = 2
camel = densities.Camel(D)
camel_1d = densities.Camel(1)
x = np.linspace(0, 1, 100)
y = camel_1d.pdf(x)

In [ ]:
proposal_weights = np.array([.5, .5])
channels = MultiChannel([
    densities.Gaussian(D, mu=1/3, cov=.005),
    densities.Gaussian(D, mu=2/3, cov=.005)],
    proposal_weights)

importance_sample = channels.rvs(10000)

In [ ]:
importance_df = pd.DataFrame(importance_sample)

In [ ]:
plt.figure()
grid = sns.jointplot(x=0, y=1, data=importance_df, kind="hex", marginal_kws={"norm_hist" : True})
grid.ax_marg_x.axes.plot(x, y, 'r-')
grid.ax_marg_y.axes.plot(y, x, 'r-')

In [ ]:
hmc = hamiltonian.HamiltonianUpdate(camel, densities.Gaussian(D, .07), 10, .005)
#start = met.sample(5000, np.random.rand(5)).data[-1] # warmup
start = [1/3, 1/3]
hmc_sample = hmc.sample(10000, start)

In [ ]:
hmc_df = pd.DataFrame(hmc_sample.data)

In [ ]:
plt.figure
grid = sns.jointplot(x=0, y=1, data=hmc_df, kind="hex", marginal_kws={"norm_hist" : True})
grid.ax_marg_x.axes.plot(x, y, 'r-')
grid.ax_marg_y.axes.plot(y, x, 'r-')

In [ ]:
spherical_hmc = hamiltonian.StaticSphericalHMC(camel, .03, .03, 30, 30)
start = 2*[1/3]
spherical_hmc_sample = spherical_hmc.sample(10000, start)

In [ ]:
spherical_hmc_df = pd.DataFrame(spherical_hmc_sample.data)

In [ ]:
fig, ax = plt.subplots(1, 2, sharey='row', figsize=(10,5))
#ax1 = fig.add_subplot(1, 2, 1)
ax[0].plot(x, y, 'r-')
sns.distplot(spherical_hmc_df[0], ax=ax[0], kde=False, norm_hist=True, hist_kws={"weights" : spherical_hmc_sample.weights})
#ax2 = fig.add_subplot(1, 2, 2)
ax[1].plot(x, y, 'r-')
sns.distplot(spherical_hmc_df[1], ax=ax[1], kde=False, norm_hist=True, hist_kws={"weights" : spherical_hmc_sample.weights})

In [ ]:
util.effective_sample_size(spherical_hmc_sample, spherical_hmc_sample.mean, spherical_hmc_sample.variance)

In [ ]:
# resample
resamp_idx = np.random.choice(10000, 10000, p=spherical_hmc_sample.weights/spherical_hmc_sample.weights.sum())
spherical_hmc_resample = MarkovSample(data=spherical_hmc_sample.data[resamp_idx])
spherical_hmc_resample_df = pd.DataFrame(spherical_hmc_resample.data)

In [ ]:
fig, ax = plt.subplots(1, 2, sharey='row', figsize=(10,5))
#ax1 = fig.add_subplot(1, 2, 1)
ax[0].plot(x, y, 'r-')
sns.distplot(spherical_hmc_resample_df[0], ax=ax[0], kde=False, norm_hist=True)
#ax2 = fig.add_subplot(1, 2, 2)
ax[1].plot(x, y, 'r-')
sns.distplot(spherical_hmc_resample_df[1], ax=ax[1], kde=False, norm_hist=True)

In [ ]:
plt.figure()
grid = sns.jointplot(x=0, y=1, data=spherical_hmc_resample_df, kind="hex", marginal_kws={"norm_hist" : True})
grid.ax_marg_x.axes.plot(x, y, 'r-')
grid.ax_marg_y.axes.plot(y, x, 'r-')

In [ ]:
def adapt_schedule(x):
    return x<1000
    
adaptive_spherical_hmc = hamiltonian.DualAveragingSphericalHMC(camel, 1., adapt_schedule)
start = 2*[1/3]
adaptive_spherical_hmc_sample = adaptive_spherical_hmc.sample(2000, start)

Event 1	(batch acceptance rate: 0.000000)
Event 2	(batch acceptance rate: 0.000000)	max sequence length: 3
Event 3	(batch acceptance rate: 0.000000)	max sequence length: 4
Event 4	(batch acceptance rate: 0.000000)	max sequence length: 5
Event 5	(batch acceptance rate: 0.000000)	max sequence length: 6
Event 6	(batch acceptance rate: 0.000000)	max sequence length: 7
Event 7	(batch acceptance rate: 0.000000)	max sequence length: 8
Event 8	(batch acceptance rate: 0.000000)	max sequence length: 9
Event 9	(batch acceptance rate: 0.000000)	max sequence length: 10
